<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-255-Data_Mining/blob/main/Assignment_6/Apache_Beam_Feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rutujapatil2","key":"824684e9eb529f96eb7d30f83bfdf6da"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c ga-customer-revenue-prediction

100% 1.56G/1.56G [00:13<00:00, 151MB/s]
100% 1.56G/1.56G [00:13<00:00, 123MB/s]


In [5]:
!unzip ga-customer-revenue-prediction.zip

Archive:  ga-customer-revenue-prediction.zip
  inflating: sample_submission.csv   
  inflating: sample_submission_v2.csv  
  inflating: test.csv                
  inflating: test_v2.csv             
  inflating: train.csv               
  inflating: train_v2.csv            


In [6]:
!pip install apache-beam[gcp]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.7/184.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import apache_beam as beam

class ParseCSVLine(beam.DoFn):
    def process(self, element):
        import csv
        for row in csv.reader([element]):
            return [{'fullVisitorId': row[0], 'totals.transactionRevenue': float(row[-1]) if row[-1] else 0}]

class CalculateRevenue(beam.PTransform):
    def expand(self, input):
        return (input
                | 'Filter Revenue' >> beam.Filter(lambda record: record['totals.transactionRevenue'] > 0)
                | 'Sum Revenue' >> beam.Map(lambda record: (record['fullVisitorId'], record['totals.transactionRevenue']))
                | 'Total Revenue per Customer' >> beam.CombinePerKey(sum))

input_file = 'train.csv'  # path to the dataset

with beam.Pipeline() as pipeline:
    results = (pipeline
               | 'Read CSV' >> beam.io.ReadFromText(input_file, skip_header_lines=1)
               | 'Parse CSV' >> beam.ParDo(ParseCSVLine())
               | 'Calculate Total Revenue' >> CalculateRevenue()
               | 'Print Results' >> beam.Map(print))


('Organic Search', 567278568658022.0)
('Referral', 155780651263159.0)
('Paid Search', 37640596592587.0)
('Affiliates', 24374097997421.0)
('Direct', 212584067083562.0)
('Display', 9301102047649.0)
('Social', 334793274605924.0)
('(Other)', 178934435839.0)


In [2]:
import apache_beam.transforms.window as window
import time

with beam.Pipeline() as pipeline:
    windowed_data = (pipeline
                     | 'Create Stream' >> beam.Create([{'fullVisitorId': '1001', 'totals.transactionRevenue': 300.0, 'timestamp': time.time()},
                                                       {'fullVisitorId': '1002', 'totals.transactionRevenue': 150.0, 'timestamp': time.time() - 3600},
                                                       {'fullVisitorId': '1001', 'totals.transactionRevenue': 50.0, 'timestamp': time.time()}])
                     | 'Add Timestamps' >> beam.Map(lambda x: beam.window.TimestampedValue(x, x['timestamp']))
                     | 'Window' >> beam.WindowInto(window.FixedWindows(60 * 60))  # 1-hour windows
                     | 'Revenue per Window' >> beam.Map(lambda x: (x['fullVisitorId'], x['totals.transactionRevenue']))
                     | 'Sum Revenue per Window' >> beam.CombinePerKey(sum)
                     | 'Print Results' >> beam.Map(print))


('1001', 350.0)
('1002', 150.0)


In [3]:
class ExtractCustomerRevenue(beam.DoFn):
    def process(self, element):
        customer_id = element['fullVisitorId']
        revenue = element['totals.transactionRevenue']
        yield (customer_id, revenue)

with beam.Pipeline() as pipeline:
    results = (pipeline
               | 'Create Data' >> beam.Create([{'fullVisitorId': '1001', 'totals.transactionRevenue': 300.0},
                                               {'fullVisitorId': '1002', 'totals.transactionRevenue': 150.0}])
               | 'Extract Customer Revenue' >> beam.ParDo(ExtractCustomerRevenue())
               | 'Print Results' >> beam.Map(print))


('1001', 300.0)
('1002', 150.0)


In [4]:
import time
import apache_beam as beam
import apache_beam.transforms.window as window

def generate_data():
    yield {'fullVisitorId': '1001', 'totals.transactionRevenue': 200.0, 'timestamp': time.time()}
    time.sleep(2)
    yield {'fullVisitorId': '1002', 'totals.transactionRevenue': 150.0, 'timestamp': time.time()}
    time.sleep(2)
    yield {'fullVisitorId': '1001', 'totals.transactionRevenue': 50.0, 'timestamp': time.time()}

with beam.Pipeline() as pipeline:
    streaming_data = (pipeline
                      | 'Generate Data' >> beam.Create(generate_data()) # Use beam.Create to ingest data from the generator
                      | 'Add Timestamps' >> beam.Map(lambda x: beam.window.TimestampedValue(x, x['timestamp']))
                      | 'Window' >> beam.WindowInto(window.FixedWindows(60 * 60))  # 1-hour windows
                      | 'Print Results' >> beam.Map(print))

{'fullVisitorId': '1001', 'totals.transactionRevenue': 200.0, 'timestamp': 1728160491.2627342}
{'fullVisitorId': '1002', 'totals.transactionRevenue': 150.0, 'timestamp': 1728160493.2639647}
{'fullVisitorId': '1001', 'totals.transactionRevenue': 50.0, 'timestamp': 1728160495.2659585}


In [14]:
from apache_beam.options.pipeline_options import PipelineOptions

# Enable unsafe triggers in the pipeline options
options = PipelineOptions(flags=['--allow_unsafe_triggers'])

# Define the Beam pipeline
with beam.Pipeline(options=options) as pipeline:
    windowed_with_trigger = (pipeline
                             | 'Create Data' >> beam.Create([{'fullVisitorId': '1001', 'totals.transactionRevenue': 300.0, 'timestamp': time.time()},
                                                             {'fullVisitorId': '1002', 'totals.transactionRevenue': 150.0, 'timestamp': time.time() - 3600},
                                                             {'fullVisitorId': '1001', 'totals.transactionRevenue': 50.0, 'timestamp': time.time()}])
                             | 'Add Timestamps' >> beam.Map(lambda x: beam.window.TimestampedValue(x, x['timestamp']))
                             | 'Window' >> beam.WindowInto(beam.window.FixedWindows(60 * 60),
                                                           trigger=beam.transforms.trigger.AfterProcessingTime(5),
                                                           accumulation_mode=beam.transforms.trigger.AccumulationMode.DISCARDING)
                             | 'Print Results' >> beam.Map(print))


{'fullVisitorId': '1001', 'totals.transactionRevenue': 300.0, 'timestamp': 1728162474.1621928}
{'fullVisitorId': '1002', 'totals.transactionRevenue': 150.0, 'timestamp': 1728158874.1621933}
{'fullVisitorId': '1001', 'totals.transactionRevenue': 50.0, 'timestamp': 1728162474.1621943}
